In [1]:
!pip install soynlp

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt, Mecab
from soynlp.word import WordExtractor

# 데이터
- 전처리 및 맞춤법 검사가 된 문장

In [3]:
data = pd.read_csv('/aiffel/train_1000.csv')
data.head()

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화


In [4]:
sentences = data.non_label_sentence
sentences = [sen for sen in sentences]

from soynlp.tokenizer import LTokenizer
vocab_size = 30000

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

sentences = [tokenizer.tokenize(sen) for sen in sentences]
tokenizer_tf = Tokenizer(num_words=vocab_size)
tokenizer_tf.fit_on_texts(sentences)
word_dic = tokenizer_tf.word_index
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences)
np.shape(padded)

training was done. used memory 0.507 Gbry 0.413 Gb
all cohesion probabilities was computed. # words = 1012
all branching entropies was computed # words = 26160
all accessor variety was computed # words = 26160


(4913, 433)

In [5]:
len(word_dic)

52616

In [6]:
train_data = padded
train_label = data['class']
print(len(train_data), len(train_label))

4913 4913


In [7]:
labels = {'직장 내 괴롭힘 대화':0, '기타 괴롭힘 대화':1, '갈취 대화':2, '협박 대화':3, '일반 대화': 4}
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3930 491 492
3930 491 492


# 모델 
- LSTM 사용

In [8]:
word_vector_dim = 1024
labels_size = len(labels)
hidden_size = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size//2))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(labels_size, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 1024)        30720000  
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         590336    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 30000)             1950000   
_________________________________________________________________
dropout (Dropout)            (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1

In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              
epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
16/16 [==============================] - 10s 277ms/step - loss: 1.3814 - accuracy: 0.3570 - val_loss: 0.8895 - val_accuracy: 0.6640
Epoch 2/10
16/16 [==============================] - 4s 230ms/step - loss: 0.5633 - accuracy: 0.7878 - val_loss: 0.5581 - val_accuracy: 0.7760
Epoch 3/10
16/16 [==============================] - 4s 228ms/step - loss: 0.1732 - accuracy: 0.9440 - val_loss: 0.6844 - val_accuracy: 0.8065
Epoch 4/10
16/16 [==============================] - 4s 238ms/step - loss: 0.0430 - accuracy: 0.9868 - val_loss: 0.8150 - val_accuracy: 0.8167
Epoch 5/10
16/16 [==============================] - 4s 231ms/step - loss: 0.0297 - accuracy: 0.9911 - val_loss: 0.9039 - val_accuracy: 0.8228
Epoch 6/10
16/16 [==============================] - 4s 237ms/step - loss: 0.0144 - accuracy: 0.9957 - val_loss: 0.9907 - val_accuracy: 0.8147
Epoch 7/10
16/16 [==============================] - 4s 237ms/step - loss: 0.0221 - accuracy: 0.9931 - val_loss: 0.9547 - val_accuracy: 0.8045
Epoch

In [28]:
pred = model.predict(test_X)

from sklearn.metrics import classification_report

pred_label = []
test_label = []

for i in range(len(pred)):
    pred_label.append((str)(np.argmax(pred[i])))
    test_label.append((str)(labels[data['class'][test_Y.index[i]]]))

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.82      0.81      0.82       110
           1       0.74      0.70      0.72       115
           2       0.86      0.70      0.77        94
           3       0.60      0.80      0.69        69
           4       0.92      0.95      0.93       104

    accuracy                           0.79       492
   macro avg       0.79      0.79      0.79       492
weighted avg       0.80      0.79      0.79       492



In [87]:
to_text = {0:'직장 내 괴롭힘 대화', 1:'기타 괴롭힘 대화', 2:'갈취 대화', 3:'협박 대화', 4:'일반 대화'}
result = pd.DataFrame([pred_label,test_label]).T
test_text = data.non_label_sentence[list(test_Y.index)]
test_text = pd.DataFrame(zip(test_text))
test_result = pd.concat([test_text, result], axis=1)
test_result.columns = ['conversation', 'pred', 'real']
error_result = test_result[test_result['pred'] != test_result['real']]
error_result['pred'] = error_result['pred'].apply(lambda x: to_text[int(x)])
error_result['real'] = error_result['real'].apply(lambda x: to_text[int(x)])
error_result

/tmp/ipykernel_992/3022590611.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_result['pred'] = error_result['pred'].apply(lambda x: to_text[int(x)])
/tmp/ipykernel_992/3022590611.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_result['real'] = error_result['real'].apply(lambda x: to_text[int(x)])


,conversation,pred,real
3,아줌마 손에 든 거 뭐예요\n장바구니인데요\n아니 장바구니 인건 아는데 뭐 들었어요...,기타 괴롭힘 대화,갈취 대화
4,뭐라고 네가 뭔데\n뭐가요\n너 나 누군지 몰라\n몰라요\n그래 너 잘 만났다 ...,기타 괴롭힘 대화,협박 대화
8,아오 어디서 썩은 냄새가 나냐 야 창문 열어 구토 나오겠네\n그러네요 자꾸 어디서 ...,기타 괴롭힘 대화,직장 내 괴롭힘 대화
11,야 너네 집에 키위 농장한다며\n어 응\n내가 키위 진짜 좋아하는데 좀 얻어먹어보자...,직장 내 괴롭힘 대화,갈취 대화
17,저 오늘 조퇴하고 싶은데요 선생님\n안돼\n몸이 안 좋아요\n네가 좋은 적 있었니...,갈취 대화,기타 괴롭힘 대화
...,...,...,...
462,이때까지 돈 가져와\n나 그만큼 돈 없어\n없으면 빌리던지\n누구한테 빌려\n친구 ...,협박 대화,갈취 대화
469,자네는 회사 놀러 다니나\n무슨 말씀이신지\n맨날 그렇게 꾸며 입고 화장할 시간에 ...,협박 대화,직장 내 괴롭힘 대화
470,어이 아저씨 그거 밥 어디서 받은 거예요\n저기 노인정에 대학생들 봉사활동 왔 씨...,협박 대화,갈취 대화
480,야 라면 하나 끓여봐\n아 형이 끓여\n뭐 이 새끼가 미쳐 돌았냐\n아 왜 맨 나...,갈취 대화,기타 괴롭힘 대화


In [89]:
Sam = error_result[error_result['real'] == '직장 내 괴롭힘 대화'].sample(n=2)
id = Sam.index
Sam[['pred', 'real']]

,pred,real
415,기타 괴롭힘 대화,직장 내 괴롭힘 대화
290,협박 대화,직장 내 괴롭힘 대화


In [90]:
Sam['conversation'][id[0]]

'승인해주십시오\n진행하던 사건 손 떼고 세미나 준비나 잘 해\n그럴 순 없는데요\n야 누가 지금 너랑 흥정하자 든\n죄송합니다\n하라면 해\n그분 막아야 합니다\n너 진짜  말 안 들을래  정말 찍히게 해줘\n잘라내야 할 게 무엇인지 다시 검토해주십시오\n꼴도 보기 싫으니까 나가'

In [91]:
Sam['conversation'][id[1]]

'이번에 제가 낸 프로젝트 어때요\n그냥 솔직히 말하세요 안 불편하세요\n네  뭐가요\n그거 제 아이디어잖아요\n무슨 소리세요\n제가 낸 아이디어 가져가서 쓰신 거잖아요\n무슨 너 아이디어야 말은 제대로 해  실적 낮춰줘\n네\n네가 뭐라 하던 난 널 망가뜨릴 수 있어 그냥 조용히 있어\n회사에 말할게요 저도 더 이상은 못 참겠네요'

In [93]:
Sam = error_result[error_result['real'] == '기타 괴롭힘 대화'].sample(n=2)
id = Sam.index
Sam[['pred', 'real']]

,pred,real
17,갈취 대화,기타 괴롭힘 대화
480,갈취 대화,기타 괴롭힘 대화


In [94]:
Sam['conversation'][id[0]]

'저 오늘 조퇴하고 싶은데요  선생님\n안돼\n몸이 안 좋아요\n네가 좋은 적 있었니  틱장애도 있고\n그건  배가 진짜 아파요\n공부도 못하는 게 아픈데도 많네\n선생님\n그냥 보건실 가\n어머니가 오신다고 했는데\n유난이다 진짜'

In [95]:
Sam['conversation'][id[1]]

'야 라면 하나 끓여봐\n아 형이 끓여\n뭐  이 새끼가 미쳐 돌았냐\n아 왜 맨 나 내가 끓여\n정신 차려라 뚝배기 깨지고 싶냐\n엄마한테 말한다\n말해라 지금 엄마도 없는데 네가 요새 제정신이 아닌 것 같아서 형이 정신 건강 시켜줘야겠다\n아하지 마\n뭘 하지 마 병신 새끼가 형한테 개기고\n끓여올게'

In [96]:
Sam = error_result[error_result['real'] == '갈취 대화'].sample(n=2)
id = Sam.index
Sam[['pred', 'real']]

,pred,real
175,직장 내 괴롭힘 대화,갈취 대화
122,협박 대화,갈취 대화


In [97]:
Sam['conversation'][id[0]]

'이번에 승마 대회에 나가는데 말이 필요해요\n말이요  어떤 말이면 될까요  실장님\n우리 딸이 탈거니까 제주도에서 나고 자란 최고급 말로 부탁해요\n알겠습니다  그런데 저희 회사에 돈이 좀 부족해서 그런데 언제까지 마련하면 될까요\n김 대표  요즘 배가 많이 부른 가봐\n네  아닙니다  제가 말실수를 했습니다\n알면 됐어  이달 말까지 부탁해\n네  알겠습니다  더 필요한 것은 없으세요\n다른 건 괜찮아\n네'

In [98]:
Sam['conversation'][id[1]]

'우와 이거 예쁘다  남자친구랑 맞췄어\n이거 커플링이야\n이거 진짜 금 같아\n맞아 이거 24로 맞춘 거야\n와 역시 나 이것 좀 빌려주라  나 누구한테 자랑하고 싶어\n이거 커플링인데    이건 진짜 안돼\n야 너 남자친구 만나기 전에 돌려줄게 보여줄 사람 있단 말이야  다시 잘 만나기 전에 돌려줄 테니가 걱정 마\n그래도 안돼  이거 남자친구 알면 난리 나\n야 너랑 내가 말을 안 하는데 누가 알아  우리 둘만 조용히 입 다물면 아무도 몰라  그러니까 남자친구 만나기 전까지 내가 좀 쓴다\n빨리 돌려줘야 돼'

In [99]:
Sam = error_result[error_result['real'] == '협박 대화'].sample(n=2)
id = Sam.index
Sam[['pred', 'real']]

,pred,real
213,직장 내 괴롭힘 대화,협박 대화
261,기타 괴롭힘 대화,협박 대화


In [100]:
Sam['conversation'][id[0]]

'네가 먼저 끼어들었잖아\n지금 누구 보러 너라고 지칭하십니까\n이 새끼가 인성 있는척하네 네가 내 길 방해 안 했나\n아닙니다 전 여유를 두고 들어왔습니다 비켜주시죠\n고마이 치아 다 뽑아가 피 철철 나면서 생 마감하게 해주까\n선생님 이러지 말고 비켜주시죠\n말길을 못 알아듣네 영영 눈 못 뜨게 해준다는 말인데\n죄송합니다 제가 다 죄송합니다\n표정이 영 맘에 들지 않아\n제가 표정이 안 좋았다면 죄송합니다 좋은 인상으로 바꾸겠습니다 이제 그만둬주세요'

In [101]:
Sam['conversation'][id[1]]

'저기요 할아버지 여기 버스 안이기도 한데 환기도 잘 안되니까 마스크 좀 써주세요\n네가 뭔데 나보고 쓰라 말아야  나한테 뭐 해준 거 있어\n그게 아니고 이제 버스에서 원래 마스크 안 쓰면 내리시거나 벌금 내셔야 해요\n그래서 뭐 어쩌라고  지금 나 내쫓을 수 있는 사람 있어\n그럼 여기서 내리세요 제가 내려드릴게요  공공장소인데 남한테 피해주면 안 되시죠\n너 내가 죽여버릴 거야  각오해  마스크  그거 평생 안 쓰게 만들어줄게\n그게 뭔 소리세요  그런 말 함부로 하시면 안 되죠  아무리 연세가 많으시다고 해도\n그니까 애초에 왜 내 신경을 건드려  내가 지금은 이렇게 조용하지만 어릴 적에는 여럿 패고 다녔어  오랜만에 힘 좀 써야 하네\n협박하시면 신고하실 거고요  마스크 하나 써달라고 했다고 그렇게 말씀하시지 마세요\n내가 뭐 어떻게 말하던 무슨 상관인가  실제로 죽일 것도 아닌데  오늘 저녁에 죽을 거 같을 때 신나게 신고하던지 끌끌'

In [102]:
Sam = error_result[error_result['real'] == '일반 대화'].sample(n=2)
id = Sam.index
Sam[['pred', 'real']]

,pred,real
41,기타 괴롭힘 대화,일반 대화
211,협박 대화,일반 대화


In [103]:
Sam['conversation'][id[0]]

'외국은 또 무슨 이슈가 있니\n그러게 외국에는 아무 일 없냐\n헐 빌게이츠도 14년 전인가 유부남일 때 여직원한테 만나자고 구애 문자 보냈다더라\n아니 그거까지 너가 어떻게 아는데\n아니 다 알지 내가\n경영진이 경고도 했다던디\n정말\n기사 나온 거야\n응응 아니\n유부녀가 자기가 대표라는 권위를 이용해서 여직원한테 구애를 하는 거잖어\n그러게\n근데 사람 다 똑같다\n외국에서도 그런 일이 일어나는구나\n사람 사는 곳은 다 그런가 봐\n그러게\n좀 뜨고 이러면 사람들 괜히 자기 권력 이용함\n맞아\n잠잠코 성실히 살면 쭉 행복할 텐데 왜 굳이 그럴까\n그러게 너는 그러지 마라 아\n우선 떠야 하는 거 아닐까\n그게 먼저긴 하지\n미안하다\n우선순위 잘 정하고 말해줘\n그래 그래야겠다\n아 피부에 뭐남'

In [104]:
Sam['conversation'][id[1]]

'안녕하세요 입니다 무엇을 도와드릴까요\n네 여기 인데요\n여기 악취가 너무 심해서요 처리 좀 부탁드립니다\n네 악취 말씀이세요\n이곳은 현재 악취에 대한 민원이 많이 들어온 상태라서요\n저희도 악취를 줄이기 위해 노력하고 있습니다\n어떤 노력이요 하나도 줄지를 않는데요\n구체적으로 에서 부지경계에 설치한 센서의 농도가 높아지면요\n공무원이 방문하여 안개분무시설 사용 주기를 단축하고 있구요\n그게 아니면 탈취제를 뿌려 악취 농도를 낮게 만들고 있습니다\n그것밖에 없는 거예요\n아뇨 또한 환경감시단이 수시로 인근을 순찰할 계획이구요\n농장주를 만나 악취저감과 관련된 조치를 취할 예정입니다\n그렇게 되면 농장주도 관리에 더 신경을 쓸 것이구요\n악취 농도를 영으로 만들진 못하겠지만 점차 나아질 것이라 생각합니다\n네 알겠습니다 일단 뭐 하고 계신 것 같으니까 기다려보겠습니다\n네 항상 노력하겠습니다\n추가 질문이나 설명이 필요하신 경우 으로 연락 주시기바랍니다'